## BRONZE TO SILVER LAYER

### Silver Layer - Stock Events History


In [1]:
# Import necessary libraries and utility functions
import pandas as pd
from common.utilities import (
global_path,
replace_punctuation_from_columns,
check_files_availability,
logger,
)

### Data Processing

This section handles the collection of data from the Bronze layer.
- Files are checked for availability.
- Data is read from each file, processed, and stored in a list of DataFrames.
- Process stock event data from the Bronze layer and transform it for the Silver layer.

In [2]:
# Initialize an empty list to store DataFrames
df_stock_events_list = []

# Generate file paths for available CSV files in the Bronze layer
file_paths = check_files_availability(
    global_path.stockdata_bronze_layer_path, file_pattern="*.csv"
)

# Process each CSV file found in the Bronze layer
for file_path in file_paths:
    logger.info(f"Processing file: {file_path}")
    
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Extract stock name from file path and add it as a column
    df["stock_name"] = file_path.name.split(".")[0].upper().strip()

    # Append the DataFrame to the list
    df_stock_events_list.append(df)

# Concatenate all DataFrames into a single DataFrame
df = pd.concat(df_stock_events_list, ignore_index=True)

# Harmonize column names to ensure consistency
df = replace_punctuation_from_columns(df)

# Remove columns that contain only NA values
df.dropna(how="all", axis=1, inplace=True)

2024-08-07T12:50:00Z - INFO - Number of Files Detected: 92
2024-08-07T12:50:00Z - INFO - Processing file: C:\Users\prashant.tripathi\Code\Upstox\DATA\BRONZE\StockData\2020\BHAGERIA.NS.csv


2024-08-07T12:50:00Z - INFO - Processing file: C:\Users\prashant.tripathi\Code\Upstox\DATA\BRONZE\StockData\2020\BPCL.NS.csv
2024-08-07T12:50:00Z - INFO - Processing file: C:\Users\prashant.tripathi\Code\Upstox\DATA\BRONZE\StockData\2020\GOLDBEES.NS.csv
2024-08-07T12:50:00Z - INFO - Processing file: C:\Users\prashant.tripathi\Code\Upstox\DATA\BRONZE\StockData\2020\IDEA.NS.csv
2024-08-07T12:50:00Z - INFO - Processing file: C:\Users\prashant.tripathi\Code\Upstox\DATA\BRONZE\StockData\2020\INFY.NS.csv
2024-08-07T12:50:00Z - INFO - Processing file: C:\Users\prashant.tripathi\Code\Upstox\DATA\BRONZE\StockData\2020\IRCTC.NS.csv
2024-08-07T12:50:01Z - INFO - Processing file: C:\Users\prashant.tripathi\Code\Upstox\DATA\BRONZE\StockData\2020\KPITTECH.NS.csv
2024-08-07T12:50:01Z - INFO - Processing file: C:\Users\prashant.tripathi\Code\Upstox\DATA\BRONZE\StockData\2020\MIRAE-ASSET-TAX-SAVER-DIRECT-GROWTH.MF.csv
2024-08-07T12:50:01Z - INFO - Processing file: C:\Users\prashant.tripathi\Code\Upstox

In [3]:
# Convert datetime strings to date objects for consistency
df["date"] = pd.to_datetime(df["date"]).dt.date

# Pivot the DataFrame: columns become rows
df = df.melt(
    id_vars=["date", "stock_name"],
    value_vars=["dividends", "stock_splits", "capital_gains"],
    var_name="event",
    value_name="value",
)

# Convert event names to uppercase for uniformity
df["event"] = df["event"].str.upper()

# Round numerical values to 2 decimal places
df["value"] = df["value"].fillna(0)
df["value"] = df["value"].round(2)

# Filter out rows where the value is zero or less than a threshold
df = df[df["value"] > 0]

# Sort the DataFrame by stock name, date, and event for easier analysis
df = df.sort_values(by=["stock_name", "date", "event"])

# Select relevant columns to export
df = df[["date", "stock_name", "event", "value"]]

# Save the cleaned and transformed DataFrame as a CSV file in the Silver layer
df.to_csv(global_path.stockevents_silver_file_path, index=False)
logger.info("SILVER Layer CSV file for Stock Events history successfully created at:")
logger.info(global_path.stockevents_silver_file_path.resolve())

# Log the DataFrame info for verification
df.info()

2024-08-07T12:50:01Z - INFO - SILVER Layer CSV file for Stock Events history successfully created at:
2024-08-07T12:50:01Z - INFO - C:\Users\prashant.tripathi\Code\Upstox\DATA\SILVER\StockEvents\StockEvents_data.csv


<class 'pandas.core.frame.DataFrame'>
Index: 73 entries, 158 to 20519
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        73 non-null     object 
 1   stock_name  73 non-null     object 
 2   event       73 non-null     object 
 3   value       73 non-null     float64
dtypes: float64(1), object(3)
memory usage: 2.9+ KB
